# Day 09:
## Part One

In [1]:
def expand(codes, pos):
    if pos >= len(codes):
        codes.extend([0]*(pos-len(codes) + 1))

def r(codes, pos):
    expand(codes, pos)
    return codes[pos]
        
def w(codes, pos, offset, val):
    write_pos = get_out_pos(codes, pos, offset)
    expand(codes, write_pos)
    codes[write_pos] = val

In [2]:
def get_params(codes, pos, num):
    global base
    res = []
    modes = list(str(codes[pos])[:-2])
    for arg in codes[pos+1:pos+num+1]:
        if len(modes) > 0:
            m = int(modes.pop())
        else:
            m = 0
        if m == 0:
            res.append(r(codes, arg))
        elif m == 1:
            res.append(arg)
        elif m == 2:
            res.append(r(codes, base + arg))
    return res if len(res) > 1 else res[0]

def get_out_pos(codes, pos, offset):
    modes = list(str(codes[pos])[:-2])
    #print(modes, pos, offset)
    if len(modes) >= offset and modes[0] == "2":
        return base + codes[pos+offset]
    return codes[pos + offset]

def add(codes, pos):
    a,b = get_params(codes, pos, 2)
    w(codes, pos, 3, a + b)
    return pos + 4

def multiply(codes, pos):
    a,b = get_params(codes, pos, 2)
    w(codes, pos, 3, a * b)
    return pos + 4

def inpt(codes, pos, inp):
    w(codes, pos, 1, inp.pop())
    return pos + 2

def outpt(codes, pos):
    a = get_params(codes, pos, 1)
    return pos + 2, a

def halt(codes, pos):
    return float("inf")

def jump_if_true(codes, pos):
    a,b = get_params(codes, pos, 2)
    if a != 0:
        return b
    return pos+3

def jump_if_false(codes, pos):
    a,b = get_params(codes, pos, 2)
    if a == 0:
        return b
    return pos+3

def less_than(codes, pos):
    a,b = get_params(codes, pos, 2)
    w(codes, pos, 3, int(a < b))
    return pos+4
        
def equals(codes, pos):
    a,b = get_params(codes, pos, 2)
    w(codes, pos, 3, int(a==b))
    return pos+4

def adjust_base(codes, pos):
    global base
    a = get_params(codes, pos, 1)
    base += a
    return pos + 2

ops = {
    1: add,
    2: multiply,
    #3: inpt,
    #4: outpt,
    5: jump_if_true,
    6: jump_if_false,
    7: less_than,
    8: equals,
    9: adjust_base,
    99: halt
}

In [3]:
from collections import deque
class Intcode:
    def __init__(self, program, inp=[]):
        self.codes = [int(i) for i in program.split(',')]
        self.inp = deque(inp)
        self.base = 0
        self.pos = 0
        self.ops = {
            1: self._add,
            2: self._multiply,
            #3: self._inpt,
            #4: self._outpt,
            5: self._jump_if_true,
            6: self._jump_if_false,
            7: self._less_than,
            8: self._equals,
            9: self._adjust_base,
            99: self._halt
        }
        
    def run(self):
        while self.pos < len(self.codes):
            opcode = self.codes[self.pos] % 100

            if opcode == 4:
                self.pos, out = self._outpt()
                yield out
            elif opcode == 3:
                self.pos = self._inpt()
            else:
                self.pos = self.ops[opcode]()
                
    def _get_params(self, num):
        res = []
        modes = list(str(self.codes[self.pos])[:-2])
        for arg in self.codes[self.pos+1:self.pos+num+1]:
            if len(modes) > 0:
                m = int(modes.pop())
            else:
                m = 0
            if m == 0:
                res.append(self._r(arg))
            elif m == 1:
                res.append(arg)
            elif m == 2:
                res.append(self._r(self.base + arg))
        return res if len(res) > 1 else res[0]

    def _get_out_pos(self, pos, offset):
        modes = list(str(self.codes[pos])[:-2])
        if len(modes) >= offset and modes[0] == "2":
            return self.base + self._r(pos+offset)
        return self.codes[pos + offset]
    
    def _expand(self, pos):
        if pos >= len(self.codes):
            self.codes.extend([0]*(pos-len(self.codes) + 1))

    def _r(self, pos):
        self._expand(pos)
        return self.codes[pos]

    def _w(self, pos, offset, val):
        write_pos = self._get_out_pos(pos, offset)
        self._expand(write_pos)
        self.codes[write_pos] = val
        
    def _outpt(self):
        a = self._get_params(1)
        return self.pos + 2, a
    
    def _inpt(self):
        self._w(self.pos, 1, self.inp.popleft())
        return self.pos + 2
    
    def _add(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, a + b)
        return self.pos + 4
    
    def _multiply(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, a * b)
        return self.pos + 4
    
    def _halt(self):
        return float("inf")

    def _jump_if_true(self):
        a,b = self._get_params(2)
        if a != 0:
            return b
        return self.pos+3

    def _jump_if_false(self):
        a,b = self._get_params(2)
        if a == 0:
            return b
        return self.pos+3

    def _less_than(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, int(a < b))
        return self.pos+4

    def _equals(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, int(a==b))
        return self.pos+4

    def _adjust_base(self):
        a = self._get_params(1)
        self.base += a
        return self.pos + 2

In [1]:
import intcode

In [3]:
with open("input/9_1.txt") as f:
    program = f.read().strip()
m = intcode.VM(program, [1])
m.run()

4288078517

In [4]:
m = intcode.VM(program, [2])
m.run()

69256